# Request VC from Authority

This notebook is used in combination with notebook `01_issue_VC_city.ipynb` (see Authority agent). Break points indicate when to switch notebooks.

In [1]:
%%javascript
document.title='Manufacturer1 Agent'

<IPython.core.display.Javascript object>

#### Imports

In [2]:
%autoawait 
from aries_cloudcontroller import AriesAgentController
import os
from pprintpp import pprint
from termcolor import colored

IPython autoawait is `on`, and set to use `asyncio`


#### Initialize `Manufacturer1` Agent Controller

In [3]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")
agent_controller = AriesAgentController(admin_url,api_key)

Initialising a controller with admin api at http://manufacturer1-agent:3021 and an api key of adminApiKey


## 1 – Init Webhook Server and register Event Listeners for it
#### 1.1 – Start webhook server
Start a webhook server to be able to communicate with other agents

In [4]:
# Setup
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

# Listen on webhook server
await agent_controller.init_webhook_server(webhook_host, webhook_port)
print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Listening for webhooks from agent at http://0.0.0.0:3010


#### 1.2 – Define listeners
Define listeners that are triggered when something happens on the webhook server

In [5]:
# Receive connection messages
def connections_handler(payload):
    state = payload['state']
    connection_id = payload["connection_id"]
    their_role = payload["their_role"]
    routing_state = payload["routing_state"]
    rfc_state = payload["rfc23_state"]
    
    print("----------------------------------------------------------")
    print("Connection Webhook Event Received")
    print("Connection ID : ", connection_id)
    print("State : ", state)
    print("Routing State : {routing} ({rfc})".format(routing=routing_state, rfc=rfc_state))
    if 'their_label' in payload: 
        print(f"Connection with : ", payload['their_label'])
    print("Their Role : ", their_role)
    print("----------------------------------------------------------")

    if state == "invitation":
        # Your business logic
        print("invitation")
    elif state == "request":
        # Your business logic
        print("request")

    elif state == "response":
        # Your business logic
        print("response")
    elif state == "active":
        # Your business logic
        print(colored("Connection ID: {0} is now active.".format(connection_id), "green", attrs=["bold"]))

        
## YOUR LISTENERS HERE
def holder_handler(payload):
    connection_id = payload['connection_id']
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    print("\n---------------------------------------------------\n")
    print("Handle Issue Credential Webhook")
    print(f"Connection ID : {connection_id}")
    print(f"Credential exchange ID : {exchange_id}")
    print("Agent Protocol Role : ", role)
    print("Protocol State : ", state )
    print("\n---------------------------------------------------\n")
    print("Handle Credential Webhook Payload")
    
    if state == "offer_received":
        print("Credential Offer Recieved")
        proposal = payload["credential_proposal_dict"]
        print("The proposal dictionary is likely how you would understand and display a credential offer in your application")
        print("\n", proposal)
        print("\n This includes the set of attributes you are being offered")
        attributes = proposal['credential_proposal']['attributes']
        print(attributes)
        ## YOUR LOGIC HERE
    elif state == "request_sent":
        print("\nA credential request object contains the commitment to the agents master secret using the nonce from the offer")
        ## YOUR LOGIC HERE
    elif state == "credential_received":
        print("Received Credential")
        ## YOUR LOGIC HERE
    elif state == "credential_acked":
        ## YOUR LOGIC HERE
        credential = payload["credential"]
        print("Credential Stored\n")
        print(credential)
        
        print("\nThe referent acts as the identifier for retrieving the raw credential from the wallet")
        # Note: You would probably save this in your application database
        credential_referent = credential["referent"]
        print("Referent", credential_referent)
        
        
def messages_handler(payload):
    connection_id = payload["connection_id"]
    agent_controller.messaging.send_message(connection_id, "This is a response from Bob")
    print("Handle message", connection_id)
    pprint(payload)

In [6]:
# Init listener list 
listeners = []

# Add listeners defined in previous cell
connection_listener = {"handler": connections_handler, "topic": "connections"}
listeners.append(connection_listener)

holder_listener = {"topic": "issue_credential", "handler": holder_handler}
listeners.append(holder_listener)

message_listener = {"handler": messages_handler, "topic": "basicmessages"}
listeners.append(message_listener)

#### 1.3 – Register listeners with `agent_controller`

In [7]:
agent_controller.register_listeners(listeners)

## 2 – Establish a connection with Authority agent
A connection with the credential issuer (i.e., the authority agent) must be establieshed before a VC can be received. In this scenario, the agent requests a connection with the Authority to be certified as an official city agency. Thus, the city agent sends an invitation to the Authority.

### 2.1 Create invitation to Authority agent

In [8]:
# Setup for connection with Authority agent
alias = None
auto_accept = "true" # Accept response of Authority agent right away
public = "false" # Do not use public DID
multi_use = "false" # Invitation is only for one invitee

# Await response
invitation_response = await agent_controller.connections.create_invitation(alias, auto_accept, public, multi_use,)

# Keep connection_id to use it in other protocols (e.g., messaging)
connection_id = invitation_response["connection_id"]

----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  32200c7f-0fea-4c78-ae7b-57f4ad9217b0
State :  invitation
Routing State :  none
Their Role :  invitee
----------------------------------------------------------
invitation


### 2.2 – Send invitation with Authority
Sharing the invitation can be done via e.g., phone, zoom, E-Mail or QR-code. In this scenario, please copy and paste the invitation into `01_issue_VC_city.ipynb` of the Authority agent (see Step 2.1)

In [9]:
# Get and print invitation
invitation = invitation_response["invitation"]
print("Copy the following dict and past it into the Authority agency")
pprint(invitation)

Copy the following dict and past it into the Authority agency
{
    '@id': '149a1be3-e17d-44d5-92f4-b40ddb629649',
    '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
    'label': 'Manufacturer1',
    'recipientKeys': ['8x5QM4KWByjUVdqbdTBdv2a1zABkH6uC4j9Ab9yhX2ng'],
    'serviceEndpoint': 'https://c1b8e3f90d1a.ngrok.io',
}
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  32200c7f-0fea-4c78-ae7b-57f4ad9217b0
State :  request
Routing State :  none
Their Role :  invitee
----------------------------------------------------------
request
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  32200c7f-0fea-4c78-ae7b-57f4ad9217b0
State :  response
Routing State :  none
Their Role :  invitee
----------------------------------------------------------
response


**BREAK POINT:** Please switch to agent `Authority`, open `01_issue_VC_city.ipynb`, and continue with Step 2

---

### 2.3 – Sent trust ping to establish and activate connection

In [10]:
await agent_controller.messaging.trust_ping(connection_id, "send trust ping")

{'thread_id': '3f3efe05-98d2-4c2f-8861-e9c0ddb1cee2'}

----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  32200c7f-0fea-4c78-ae7b-57f4ad9217b0
State :  active
Routing State :  none
Their Role :  invitee
----------------------------------------------------------
Connection ID: 32200c7f-0fea-4c78-ae7b-57f4ad9217b0 is now active.


## 3 – Request VC from `Authority` agent
### 3.1 – Message `Authority` to request a VC

In [11]:
basic_message = "Hello Authority agent"
await agent_controller.messaging.send_message(connection_id, basic_message)

{}

/tmp/ipykernel_16/3503404827.py:75: RuntimeWarning: coroutine 'MessagingController.send_message' was never awaited
  agent_controller.messaging.send_message(connection_id, "This is a response from Bob")


Handle message 32200c7f-0fea-4c78-ae7b-57f4ad9217b0
{
    'connection_id': '32200c7f-0fea-4c78-ae7b-57f4ad9217b0',
    'content': 'Hello Manufacturer1. Plase provide me with the following information: manufacturerName, manufacturerCountry, ManufacturerCity.',
    'message_id': '364af965-e994-4e76-b7d7-419ff01c0dca',
    'sent_time': '2021-08-09 14:21:52.649646Z',
    'state': 'received',
}


**BREAK POINT:** Go to Step 3 in the `01_issue_VC_city.ipynb` notebook of the `Authority` agent.

---

### 3.1 – Provide `Authority` with the relevant information to issue a VC

In [13]:
basic_message = '{"manufacturerName": "undisclosedName1", "manufacturerCountry": "DE", "manufacturerCity": "Munich"}'
await agent_controller.messaging.send_message(connection_id, basic_message)

{}


---------------------------------------------------

Handle Issue Credential Webhook
Connection ID : 32200c7f-0fea-4c78-ae7b-57f4ad9217b0
Credential exchange ID : c6afd888-d0a8-40d6-b6d7-46a965da1ace
Agent Protocol Role :  holder
Protocol State :  offer_received

---------------------------------------------------

Handle Credential Webhook Payload
Credential Offer Recieved
The proposal dictionary is likely how you would understand and display a credential offer in your application

 {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/propose-credential', '@id': 'caff43b9-fe3f-47a9-abb9-587db387e522', 'schema_id': 'MTXSc4YD8wixyM9ekZbmMC:2:certify-manufacturer:0.0.1', 'cred_def_id': 'MTXSc4YD8wixyM9ekZbmMC:3:CL:246687:default', 'credential_proposal': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/credential-preview', 'attributes': [{'name': 'manufacturerName', 'value': 'undisclosedName1'}, {'name': 'manufacturerCity', 'value': 'Munich'}, {'name': '

**BREAK POINT:** Return to step 

---


### 3.2 – Request VC from `Authority`'s Offer

Note: Your agent will automatically respond if ACAPY_AUTO_RESPOND_CREDENTIAL_OFFER=true flag is set in .env file of agent. Default is false.

To respond to an offer you must identify the offer using the credential_exchange_id generated for it. This is available from within the issue-credential holder handler. You could add custom logic in this loop `elif state == "request":` to handle this.

However, we will fetch the credential exchange records and **assume** this agent only has one record. Customise accordingly.

In [14]:
# Setup arguments for response
thread_id = None
state = "offer_received"
role = "prover"

records_response = await agent_controller.issuer.get_records(connection_id=connection_id,state=state)
record = records_response["results"][0]
record_id = record["credential_exchange_id"]

In [15]:
# Send request for VC
await agent_controller.issuer.send_request_for_record(record_id)


---------------------------------------------------

Handle Issue Credential Webhook
Connection ID : 32200c7f-0fea-4c78-ae7b-57f4ad9217b0
Credential exchange ID : c6afd888-d0a8-40d6-b6d7-46a965da1ace
Agent Protocol Role :  holder
Protocol State :  request_sent

---------------------------------------------------

Handle Credential Webhook Payload

A credential request object contains the commitment to the agents master secret using the nonce from the offer


{'thread_id': '3ab9d568-7182-412c-9374-3b06b1e64fb1',
 'credential_offer': {'schema_id': 'MTXSc4YD8wixyM9ekZbmMC:2:certify-manufacturer:0.0.1',
  'cred_def_id': 'MTXSc4YD8wixyM9ekZbmMC:3:CL:246687:default',
  'nonce': '1192624921435078486547423',
  'key_correctness_proof': {'c': '49818264144256857504040919915346497502971057683517596656866241068070837044558',
   'xz_cap': '1043258602390978547780576881941827126930257997684630157382635680732702336633257776074798754836300249864774391281421530206990127229422463021718422401225449963191034467077650601972787004467479889018279201419195284163321101056827112503311503227134565997010007992133396623207333495610369382516100964319510158858972111696531021012894808374807650994578163319708463801223232352986942413070635332676848522947912213963286854182044194958455783492485885889605445478800718858301627188301940995157869606188445890981976027687189991900329734037932415699198882711470676497053905499708388380997823139212494294975875744264841103846330785930170


---------------------------------------------------

Handle Issue Credential Webhook
Connection ID : 32200c7f-0fea-4c78-ae7b-57f4ad9217b0
Credential exchange ID : c6afd888-d0a8-40d6-b6d7-46a965da1ace
Agent Protocol Role :  holder
Protocol State :  credential_received

---------------------------------------------------

Handle Credential Webhook Payload
Received Credential


### 3.3 – Store received VC in wallet

This will be done automatically if the ACAPY_AUTO_STORE_CREDENTIAL=true flag is set in the .env file for this agent. Default is false.

Again you could handle this in your holder handler function in the `elif state == "credential_received":` loop.

In [16]:
credential_id = "M1-isManufacturer-VC"
store_cred_response = await agent_controller.issuer.store_credential(record_id, credential_id)


---------------------------------------------------

Handle Issue Credential Webhook
Connection ID : 32200c7f-0fea-4c78-ae7b-57f4ad9217b0
Credential exchange ID : c6afd888-d0a8-40d6-b6d7-46a965da1ace
Agent Protocol Role :  holder
Protocol State :  credential_acked

---------------------------------------------------

Handle Credential Webhook Payload
Credential Stored

{'referent': 'M1-isManufacturer-VC', 'attrs': {'isManufacturer': 'TRUE', 'manufacturerCountry': 'DE', 'manufacturerName': 'undisclosedName1', 'manufacturerCity': 'Munich'}, 'schema_id': 'MTXSc4YD8wixyM9ekZbmMC:2:certify-manufacturer:0.0.1', 'cred_def_id': 'MTXSc4YD8wixyM9ekZbmMC:3:CL:246687:default'}

The referent acts as the identifier for retrieving the raw credential from the wallet
Referent M1-isManufacturer-VC


## Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [17]:
await agent_controller.terminate()